In [34]:
import pandas as pd
import numpy as np
import h5py as h5

In [35]:
#input data
def input_data(folder):
    print('---- input cv ---- ')
    cv = pd.read_csv(folder+'/' + folder + '_video_result.csv')
    cv['pupil'] = pd.to_numeric(cv['pupil'], errors='coerce')
    cv['ratio'] = pd.to_numeric(cv['ratio'], errors='coerce')
    cv['x'] = pd.to_numeric(cv['x'], errors='coerce')
    cv['y'] = pd.to_numeric(cv['y'], errors='coerce')
    cv['ts'] = pd.to_numeric(cv['ts'], errors='coerce')
    cv['ts'] = np.around(cv['ts'], decimals=0)
    #seperate by steps
    cv = cv[cv['pupil'] > 0] #remove zeros
    
    s30_cv = cv[cv['step'] == 30]
    s31_cv = cv[cv['step'] == 31]
    s32_cv = cv[cv['step'] == 32]
    
    return cv, s30_cv, s31_cv, s32_cv

def input_eve_data(folder, step, step_num):
    print('---- input eve ---- ')
    file = folder+'/' + step + '_webcam_c.h5'
    
    left_pupil_db = pd.DataFrame(np.array(h5.File(file)['left_p']['data']))
    right_pupil_db = pd.DataFrame(np.array(h5.File(file)['right_p']['data']))
    px_2_mm = pd.DataFrame(np.array(h5.File(file)['pixels_per_millimeter']))
    gaze_right = pd.DataFrame(np.array(h5.File(file)['right_o']['data']))
    gaze_left = pd.DataFrame(np.array(h5.File(file)['left_o']['data']))
    gaze_left = gaze_left.rename(columns={0: 'x', 1: 'y', 2: 'z'})
    gaze_right = gaze_right.rename(columns={0: 'x', 1: 'y', 2: 'z'})
    scale = pd.DataFrame(np.array(h5.File(file)['pixels_per_millimeter']))
    scale = scale[0][0]
    
    #normalise input data
    #eve db
    left_pupil_db *= scale
    right_pupil_db *= scale
    
    #insert the timestamp information for the eve data
    ts_eve = attach_ts(step_num, folder)
    left_pupil_db['ts'] = ts_eve
    right_pupil_db['ts'] = ts_eve
    
    # integrate into one dataframe with the coordinate
    left_pupil_db['x'] = gaze_left['x']
    left_pupil_db['y'] = gaze_left['y']

    right_pupil_db['x'] = gaze_right['x']
    right_pupil_db['y'] = gaze_right['y']

    left_pupil_db = left_pupil_db.rename(columns={0: 'pupil'})
    right_pupil_db = right_pupil_db.rename(columns={0: 'pupil'})
    left_pupil_db = left_pupil_db[left_pupil_db['pupil'] != 0]
    right_pupil_db = right_pupil_db[right_pupil_db['pupil'] != 0]
    
    return left_pupil_db, right_pupil_db, ts_eve

In [36]:
def attach_ts(stepnum, folder):
    f = open(folder+'/step_' + stepnum +'_webcam_c.timestamps.txt','r')
    ts = []
    for line in f.readlines():
        ts.append(float(line[:-1]))
    
    f.close()
    
    start = ts[0]
    for i in range(len(ts)):
        ts[i] -= start
        ts[i] /= 1000000 #from ms to s
        ts[i] = np.around(ts[i], decimals=0)
    return ts


In [37]:
def run(step_num, step_cv_df, folder):
    step_num = str(step_num)
    step = 'step_' + step_num
    save_file = folder+'/'+step
    print(step_cv_df.head(10))
    #input eve data
    left_pupil_db, right_pupil_db, ts_eve = input_eve_data(folder, step, step_num)
    print(left_pupil_db.head(10))
    
    changes_cv_left = step_cv_df[step_cv_df['side'] == 'L'] #select individual side
    changes_cv_right = step_cv_df[step_cv_df['side'] == 'R']


In [38]:
cv_36, s30_cv36, s31_cv36, s32_cv36 = input_data('train36')
run(30, s30_cv36, 'train36') #step 30, 31, 32

---- input cv ---- 
   step     ts      x     y  pupil  ratio    sim side     d2
0    30  100.0  109.0   7.0  16.00   0.33  29.42    L  -1.00
1    30  133.0  157.0  84.0  16.00   0.33  30.25    L  -1.00
2    30  167.0  108.0  11.0  16.00   0.33  27.86    L  -1.00
3    30  200.0  106.0  15.0  20.00   0.42  26.94    L  -1.00
4    30  233.0  140.5  24.0  14.25   0.31  16.65    R  -1.00
5    30  400.0   96.0  35.0  20.00   0.35   9.58    L  -1.00
6    30  433.0  147.0  31.5  16.75   0.31   5.94    L  -1.00
7    30  500.0  145.5  30.5  16.50   0.31   6.86    L  68.31
8    30  500.0   81.0  53.0  23.00   0.46  20.29    R  68.31
9    30  533.0  145.5  29.5  16.50   0.32   7.43    L  -1.00
---- input eve ---- 
       pupil     ts          x          y
0   9.997913    0.0  65.001518 -78.580025
1   9.977455   32.0  64.510765 -78.782364
2   9.997486   68.0  64.184669 -79.252998
3  10.004448  100.0  63.921066 -79.551933
4  10.006173  132.0  63.531841 -79.610306
5  10.005296  168.0  63.018250 -80.3

In [39]:
cv_38, s30_cv38, s31_cv38, s32_cv38 = input_data('train38')

---- input cv ---- 
